In [2]:
import pandas as pd
from pysus.online_data.SIA import SIA
from datetime import datetime
import os
import gc

# Configurações
ano = 2024  # Atualizado para 2024
grupos = ['PA']  # Foco apenas nos arquivos ambulatoriais "PA"
output_dir = "/home/amcjr/pysus/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

log_file = os.path.join(output_dir, f"log_processo_{datetime.now().strftime('%d%m%y_%H%M%S')}.xlsx")

def iniciar_log():
    df_log = pd.DataFrame(columns=["Inicio", "Fim", "Duracao (horas)", "Arquivos Processados"])
    return df_log

def registrar_log(df_log, inicio, fim, arquivos_processados):
    duracao = (fim - inicio).total_seconds() / 3600.0
    duracao_str = f"{int(duracao)}:{int((duracao - int(duracao)) * 60)}"
    novo_registro = pd.DataFrame([{
        "Inicio": inicio.strftime('%d-%m-%y %H:%M:%S'),
        "Fim": fim.strftime('%d-%m-%y %H:%M:%S'),
        "Duracao (horas)": duracao_str,
        "Arquivos Processados": arquivos_processados
    }])
    df_log = pd.concat([df_log, novo_registro], ignore_index=True)
    df_log.to_excel(log_file, index=False)

def listar_arquivos_disponiveis(ano, grupos):
    sia = SIA()
    arquivos_disponiveis = []
    sia.load()  # Load necessary paths or configurations
    for grupo in grupos:
        try:
            print(f"[{datetime.now()}] Tentando carregar arquivos do grupo {grupo} para o ano {ano}.")
            arquivos = sia.get_files(group=grupo, year=ano)
            if arquivos:
                arquivos_disponiveis.extend(arquivos)
                print(f"[{datetime.now()}] Arquivos carregados para {grupo}-{ano}: {arquivos}")
            else:
                print(f"[{datetime.now()}] Nenhum arquivo encontrado para {grupo}-{ano}.")
        except Exception as e:
            print(f"[{datetime.now()}] Erro ao carregar arquivos para {grupo}-{ano}: {e}")
    return arquivos_disponiveis

def montar_dataframe_info(arquivos_disponiveis):
    # Monta um DataFrame com informações gerais sobre os arquivos disponíveis
    df_info = pd.DataFrame(arquivos_disponiveis, columns=['Arquivo'])
    df_info['Ano'] = ano
    df_info['Grupo'] = 'PA'
    return df_info

def processar_e_salvar(arquivo):
    sia = SIA()
    try:
        parquet_file_path = os.path.join(output_dir, f"{arquivo}.parquet")
        
        # Verifica se o arquivo já foi baixado
        if os.path.exists(parquet_file_path):
            print(f"[{datetime.now()}] {arquivo} já foi baixado. Pulando download.")
            return True

        print(f"[{datetime.now()}] Baixando e processando {arquivo}.")
        parquet_set = sia.download([arquivo])
        
        if not parquet_set:
            print(f"[{datetime.now()}] Não foi possível baixar o arquivo {arquivo}.")
            return False

        print(f"[{datetime.now()}] Arquivo {arquivo} salvo em {parquet_file_path}.")
        gc.collect()
        return True
        
    except Exception as e:
        print(f"[{datetime.now()}] Erro ao processar {arquivo}: {e}")
        return False

def main():
    print(f"[{datetime.now()}] Processo iniciado.")
    df_log = iniciar_log()
    inicio = datetime.now()
    
    arquivos_disponiveis = listar_arquivos_disponiveis(ano, grupos)
    total_arquivos = len(arquivos_disponiveis)
    print(f"[{datetime.now()}] Total de arquivos disponíveis: {total_arquivos}")
    
    # Montar DataFrame com informações gerais dos arquivos disponíveis
    df_info = montar_dataframe_info(arquivos_disponiveis)
    print(df_info)  # Exibe o DataFrame com informações gerais dos arquivos
    
    arquivos_processados = 0
    for i, arquivo in enumerate(arquivos_disponiveis, 1):
        sucesso = processar_e_salvar(arquivo)
        if sucesso:
            arquivos_processados += 1
        print(f"Arquivos processados: {arquivos_processados} de {total_arquivos}")
        percentual_processado = (arquivos_processados / total_arquivos) * 100
        print(f"Percentual de processamento: {percentual_processado:.0f}%")

    fim = datetime.now()
    print(f"[{datetime.now()}] Processo finalizado.")
    registrar_log(df_log, inicio, fim, arquivos_processados)

if __name__ == "__main__":
    main()


[2024-09-16 14:59:23.012461] Processo iniciado.
[2024-09-16 14:59:27.196008] Tentando carregar arquivos do grupo PA para o ano 2024.
[2024-09-16 14:59:27.391928] Arquivos carregados para PA-2024: [PAAC2401.dbc, PAAC2402.dbc, PAAC2403.dbc, PAAC2404.dbc, PAAC2405.dbc, PAAC2406.dbc, PAAC2407.dbc, PAAL2401.dbc, PAAL2402.dbc, PAAL2403.dbc, PAAL2404.dbc, PAAL2405.dbc, PAAL2406.dbc, PAAL2407.dbc, PAAM2401.dbc, PAAM2402.dbc, PAAM2403.dbc, PAAM2404.dbc, PAAM2405.dbc, PAAM2406.dbc, PAAM2407.dbc, PAAP2401.dbc, PAAP2402.dbc, PAAP2403.dbc, PAAP2404.dbc, PAAP2405.dbc, PAAP2406.dbc, PAAP2407.dbc, PABA2401.dbc, PABA2402.dbc, PABA2403.dbc, PABA2404.dbc, PABA2405.dbc, PABA2406.dbc, PABA2407.dbc, PACE2401.dbc, PACE2402.dbc, PACE2403.dbc, PACE2404.dbc, PACE2405.dbc, PACE2406.dbc, PACE2407.dbc, PADF2401.dbc, PADF2402.dbc, PADF2403.dbc, PADF2404.dbc, PADF2405.dbc, PADF2406.dbc, PADF2407.dbc, PAES2401.dbc, PAES2402.dbc, PAES2403.dbc, PAES2404.dbc, PAES2405.dbc, PAES2406.dbc, PAES2407.dbc, PAGO2401.dbc, PAGO2

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.73M/2.73M [00:00<00:00, 555MB/s]


[2024-09-16 14:59:27.501924] Arquivo PAAC2401.dbc salvo em /home/amcjr/pysus/PAAC2401.dbc.parquet.
Arquivos processados: 1 de 217
Percentual de processamento: 0%
[2024-09-16 14:59:27.631996] Baixando e processando PAAC2402.dbc.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.60M/2.60M [00:00<00:00, 770MB/s]

[2024-09-16 14:59:27.638722] Arquivo PAAC2402.dbc salvo em /home/amcjr/pysus/PAAC2402.dbc.parquet.
Arquivos processados: 2 de 217
Percentual de processamento: 1%


...

Arquivos processados: 215 de 217
Percentual de processamento: 99%
[2024-09-16 17:23:52.202870] Baixando e processando PATO2406.dbc.


PATO2406.parquet: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 417k/417k [00:14<00:00, 29.1kB/s]


[2024-09-16 17:24:08.147657] Arquivo PATO2406.dbc salvo em /home/amcjr/pysus/PATO2406.dbc.parquet.
Arquivos processados: 216 de 217
Percentual de processamento: 100%
[2024-09-16 17:24:08.270480] Baixando e processando PATO2407.dbc.


PATO2407.parquet: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 403k/403k [00:13<00:00, 29.5kB/s]

[2024-09-16 17:24:23.515858] Arquivo PATO2407.dbc salvo em /home/amcjr/pysus/PATO2407.dbc.parquet.
Arquivos processados: 217 de 217
Percentual de processamento: 100%
[2024-09-16 17:24:23.594704] Processo finalizado.
